<a href="https://colab.research.google.com/github/mrg84949-pixel/derfuu_comfyui_colab/blob/main/special/lite/derfuu_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
from IPython.display import clear_output

# 1. Монтируем диск
print("Монтируем Google Диск...")
drive.mount('/content/drive')

# 2. Настройка путей (ИСПРАВЛЕНО: exist_ok вместо exist_url)
DRIVE_PATH = '/content/drive/MyDrive/ComfyUI_Cloud'
for folder in ['checkpoints', 'loras', 'vae', 'upscale_models']:
    os.makedirs(f"{DRIVE_PATH}/models/{folder}", exist_ok=True)
os.makedirs(f"{DRIVE_PATH}/output", exist_ok=True)

# 3. Установка Lite-версии (ветка camenduru)
%cd /content
!apt -y update -qq && apt -y install -qq aria2
if not os.path.exists('ComfyUI'):
    !git clone -b v1.2.derfuu https://github.com/camenduru/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt

# 4. Установка аддонов
print("Устанавливаем плагины...")
%cd /content/ComfyUI/custom_nodes
# Удаление фона
if not os.path.exists('comfyui-rembg-node'):
    !git clone https://github.com/Vasily-67/comfyui-rembg-node
# Узлы Derfuu
if not os.path.exists('Derfuu_ComfyUI_ModdedNodes'):
    !git clone -b v1.2 https://github.com/camenduru/Derfuu_ComfyUI_ModdedNodes

!pip install -q rembg[gpu] onnxruntime-gpu

# 5. Символьные ссылки (Модели и Результаты на Диске)
print("Связываем папки с Диском...")
def link_folder(drive_folder, local_folder):
    if os.path.exists(local_folder):
        import shutil
        if os.path.islink(local_folder):
            os.unlink(local_folder)
        elif os.path.isdir(local_folder):
            shutil.rmtree(local_folder)
        else:
            os.remove(local_folder)
    os.symlink(drive_folder, local_folder)

link_folder(f"{DRIVE_PATH}/models/checkpoints", "/content/ComfyUI/models/checkpoints")
link_folder(f"{DRIVE_PATH}/models/loras", "/content/ComfyUI/models/loras")
link_folder(f"{DRIVE_PATH}/models/vae", "/content/ComfyUI/models/vae")
link_folder(f"{DRIVE_PATH}/output", "/content/ComfyUI/output")

# 6. Загрузка базовой модели (если папка пуста)
checkpoints_dir = f"{DRIVE_PATH}/models/checkpoints"
if not os.listdir(checkpoints_dir):
    print("Загружаем базовую модель...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/cheeseDaddys/resolve/main/cheeseDaddys_41.safetensors -d {checkpoints_dir} -o cheeseDaddys_41.safetensors

# 7. Запуск туннеля
print("Запускаем туннель Cloudflare...")
if not os.path.exists('/content/cloudflared-linux-amd64'):
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64
    !chmod 777 /content/cloudflared-linux-amd64

import atexit, requests, subprocess, time, re
from random import randint
from threading import Timer
from queue import Queue

def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except: pass
    output_queue.put(tunnel_url)

output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()

clear_output()
print(f"--- ЗАПУСК ЗАВЕРШЕН ---")
print(f"ССЫЛКА: {tunnel_url}")
print(f"-----------------------")

# 8. Финальный запуск
%cd /content/ComfyUI
os.environ["TORCH_ALLOW_SAFE_GLOBALS"] = "1"
!python main.py --dont-print-server --preview-method auto

--- ЗАПУСК ЗАВЕРШЕН ---
ССЫЛКА: https://insertion-oakland-championships-through.trycloudflare.com
-----------------------
/content/ComfyUI
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-01 16:48:16.001
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   5.2 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Found comfy_kitchen backend triton: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quan